In [ ]:
import math
from os import path
from argparse import ArgumentParser
import random
from pathlib import Path
import pickle

import gpytorch
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.special import logsumexp
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

from alpaca.utils.ue_metrics import uq_ll
from alpaca.ue.masks import BasicBernoulliMask, DPPMask
import alpaca.nn as ann
from alpaca.utils.model_builder import uncertainty_mode, inference_mode
from alpaca.utils.datasets.builder import build_dataset
from alpaca.utils.ue_metrics import uq_ll


def manual_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
save_dir = Path('data/regression_5')

def split_and_scale(x, y):
    # Load dat
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    # Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    y_scaler = StandardScaler()
    y_train = y_scaler.fit_transform(y_train)
    y_test = y_scaler.transform(y_test)

    return x_train, y_train, x_test, y_test, y_scaler




In [ ]:
# repeats = 1
name = 'boston_housing'

manual_seed(42)
dataset = build_dataset(name, val_split=0)
x, y = dataset.dataset('train')
N = x.shape[0] * 0.9  # train size


In [ ]:
# best_tau, best_dropout = 0.01, 0.05
# best_tau, best_dropout = select_params(x, y, N, batch_size, name, sampler)

# for i in range(repeats):
manual_seed(42)


In [ ]:

def rmse_ll(true_y, prediction, uncertainty, y_scaler):
    errors = np.abs(true_y - prediction)
    ll = uq_ll(errors, uncertainty) * y_scaler.scale_[0]
    rms_error = np.square(
        mse(
            y_scaler.inverse_transform(true_y),
            y_scaler.inverse_transform(prediction)
        )
    )
    return rms_error, ll


In [ ]:
x_train, y_train, x_test, y_test, y_scaler = split_and_scale(x, y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
pred = np.mean(y_train) * np.ones(y_test.shape)
uq = np.std(y_train)
print(rmse_ll(y_test, pred, uq, y_scaler))


class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=x_train.shape[1])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)




In [ ]:
x_train_ = torch.Tensor(x_train).cuda()
x_test_ = torch.Tensor(x_test).cuda()
y_train_ = torch.Tensor(y_train[:, 0]).cuda()
y_test_ = torch.Tensor(y_test[:, 0]).cuda()

In [ ]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train_, y_train_, likelihood)

# Find optimal model hyperparameters
model.train().cuda()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 1000
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(x_train_)
    # # Calc loss and backprop gradients

    loss = -mll(output, y_train_)
    loss.backward()
    if i % 20 == 0:
        print('Iter %d/%d - Loss: %.3f   lengthscale: %s   noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.detach().cpu().numpy()[0, :3],
            model.likelihood.noise.item()
        ))
    optimizer.step()


In [ ]:
model.likelihood.noise.item()


In [ ]:
# test_x = torch.linspace(0, 1, 51).cuda()
x_test_.cuda()
model.eval().cuda()
likelihood.eval()


In [ ]:
# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    model(x_test_)
    observed_pred = likelihood(model(x_test_))
    mean = observed_pred.mean
    lower, upper = observed_pred.confidence_region()
    uq = np.sqrt((observed_pred._covar.diag()).cpu().numpy())
    error, ll = rmse_ll(y_test[:, 0], mean.cpu().numpy(), uq, y_scaler)


In [ ]:
print(error, ll)
